In [15]:
import pandas as pd
import osmnx as ox
import numpy as np
import json
import geopandas as gpd
from shapely.geometry import Point
from sklearn.feature_extraction.text import CountVectorizer


In [ ]:
cities_pl = ['Warszawa', 'Kraków', 'Wrocław', 'Poznań', 'Szczecin']

all_nodes_pl = pd.DataFrame()
all_edges_pl = pd.DataFrame()

for city in cities_pl:
    G =ox.graph.graph_from_place(f"{city}, Poland", network_type="drive", simplify=False)
    gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)
    all_nodes_pl = pd.concat([all_nodes_pl, gdf_nodes])
    all_edges_pl = pd.concat([all_edges_pl, gdf_edges])


In [ ]:
cities_usa = ['Seattle', 'New York City']

all_nodes_usa = pd.DataFrame()
all_edges_usa = pd.DataFrame()

for city in cities_usa:
    G =ox.graph.graph_from_place(f"{city}, United States", network_type="drive", simplify=False)
    gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)
    all_nodes_usa = pd.concat([all_nodes_usa, gdf_nodes])
    all_edges_usa = pd.concat([all_edges_usa, gdf_edges])

In [ ]:
all_attributes = {}
for col in ['highway', 'access', 'junction', 'bridge', 'tunnel']:
    edges_usa = all_edges_usa[col].dropna().unique()
    edges_pl = all_edges_pl[col].dropna().unique()
    all_edges = set(np.concatenate((edges_pl, edges_usa)))
    all_attributes[f'edges_{col}'] = list(all_edges)

     
nodes_highway_usa = all_nodes_usa['highway'].dropna().unique()
nodes_highway_pl = all_nodes_pl['highway'].dropna().unique()   
all_nodes_highway = set(np.concatenate((nodes_highway_pl, nodes_highway_usa)))
all_attributes['nodes_highway'] = list(all_nodes_highway)

with open('../data/osmnx_attributes.json', 'w') as f:
    json.dump(all_attributes, f)
    

In [14]:
with open("../data/osmnx_attributes.json") as f:
    all_attributes = json.load(f)
    
num_edges_attrs = 0
for key in all_attributes.keys():
    if 'edges' in key:
        num_edges_attrs += len(all_attributes[key])
        
print('num of nodes attrs', len(all_attributes['nodes_highway']))
print('num of edges attrs', num_edges_attrs)

num of nodes attrs 21
num of edges attrs 37


In [4]:

with open("../data/osmnx_attributes.json") as f:
    all_attributes = json.load(f)

class OSMnxGraph:
    def __init__(self, gdf_events: gpd.GeoDataFrame, 
                 gdf_nodes: gpd.GeoDataFrame, 
                 gdf_edges: gpd.GeoDataFrame, 
                 start_acc_distance: float = 100,
                 y_column_name: str = 'accidents_count'
                 ):
        
        
        self.gdf_events = gdf_events
        self.gdf_nodes = gdf_nodes
        self.gdf_edges = gdf_edges
        self.start_acc_distance = start_acc_distance
        self.graph_nx = None
        self.graph_data = None
        self.y_column_name = y_column_name
        self.all_attributes = all_attributes

    
    def get_node_attrs(self):
        """Method to extract node features from OSMNx nodes GeoDataFrame. In that case retrieves highway types and street_count
        for each node. Cleaning process included applying CountVectorizer to highway column that represents highway types. Also the 
        "ref' column was removed.


        Returns:
            pd.DataFrame: cleaned dataframe that contains node features that consists of highway types and street count for each node.
        """
        if not any(col in self.gdf_nodes.columns for col in ['geometry', 'x', 'y', self.y_column_name, 'ref']):
            return self.gdf_nodes
        attrs = self.gdf_nodes.drop(['geometry', 'x', 'y', 'ref'], axis=1)
        attrs.replace('NaN', np.nan, inplace=True)
        attrs['highway'] = attrs['highway'].replace(np.nan, 'unknown')
        cols_to_add = set([item for item in all_attributes['nodes_highway'] if item not in set(attrs['highway'])])
        vectorizer = CountVectorizer(tokenizer=lambda x: x.split())
        vectorized_feature = vectorizer.fit_transform(attrs['highway'])
        df_vect_feature = pd.DataFrame(vectorized_feature.toarray(), columns=vectorizer.get_feature_names_out())
        df_vect_feature.index = attrs.index
        cleaned_df = pd.merge(attrs, df_vect_feature, left_index=True, right_index=True)
        cleaned_df.fillna(0, inplace=True)
        cleaned_df.drop(['highway', 'unknown'], axis=1, inplace=True)
        for col in cols_to_add:
            cleaned_df[col] = 0
        self.gdf_nodes = cleaned_df
        return cleaned_df
    
    def get_edge_attrs(self, convert_to_meters: bool = False):
        """Method that gets edge attributes. Includes preprocessing:
        * lanes - add default value of 2
        * maxspeed - add default value of 50 as common speed limit in urban area
        * width - add default value of 2.0 as common road width
        * other columns - add default value of "unspecified"

        If column was numeric / bool and contained list the first element was taken. If column was string type the CountVectorizer
        was applied. Then columns that were vectorized was dropped and also "ref' column was removed.


        Returns:
            pd.DataFrame: cleaned DataFrame that contains edge features
        """
        
        def _convert_mph_to_kmh(speed):
            """Method that converts speed from mph to km/h
            """
            return round(speed * 1.60934)
        
        def _check_max_speed(speed):
            """Method that checks if speed is in km/h and converts it if it is in mph
            """
            if 'mph' in speed:
                speed = speed.replace('mph', '')
                speed = int(speed)
                speed = _convert_mph_to_kmh(speed)
            return speed
        
        def _convert_to_meters(width):
            """Method that converts width from feet to meters
            """
            speed = speed.replace('ft', '') if 'ft' in speed else speed
            return round(width * 0.3048, 2)
        
        features_groups = ['highway', 'access', 'junction', 'bridge', 'tunnel']
        attrs = self.gdf_edges
        for col in attrs.columns:
            attrs = attrs.explode(col)
        if not any(col in attrs.columns for col in ['highway','osmid', 'access', 'junction', 'bridge', 'tunnel', 'geometry']):
            return attrs
        else:
            attrs = attrs.drop(['ref', 'name'], axis=1)
            attrs.replace('NaN', np.nan, inplace=True)
            attrs['width'] = attrs['width'].astype(str).apply(_convert_to_meters) if convert_to_meters else attrs['width']
            attrs['width'] = pd.to_numeric(attrs['width'], errors='coerce')
            attrs['width'] = attrs['width'].fillna(round(attrs['width'].mean(), 2)).astype(float)
            attrs['length'] = round(attrs['length'],2).astype(float)
            attrs['lanes'] = attrs['lanes'].fillna(attrs['lanes'].astype(float).mean()).astype(int)
            attrs['maxspeed'] = attrs['maxspeed'].astype(str).apply(_check_max_speed)
            attrs['maxspeed'] = attrs["maxspeed"].replace('nan', np.nan)
            attrs['maxspeed'] = attrs["maxspeed"].fillna(round(attrs['maxspeed'].astype(float).mean()))
            attrs['reversed'] = attrs['reversed'].map({True: 1, False: 0}).astype(int)
            attrs['oneway'] = attrs['oneway'].map({True: 1, False: 0}).astype(int)
            attrs = attrs.fillna("unspecified")
            vect = CountVectorizer(tokenizer=lambda x: x.split())
            idx = attrs.index
            cleaned_df = attrs.copy()
            cleaned_df.reset_index(drop=True, inplace=True)
            cleaned_df = cleaned_df.drop(['highway','osmid', 'access', 'junction', 'bridge', 'tunnel', 'geometry'], axis=1)
            for col in features_groups:                  
                vectorized_feature = vect.fit_transform(attrs[col])
                df_feature_count = pd.DataFrame(vectorized_feature.toarray(), columns=vect.get_feature_names_out())
                df_feature_count = df_feature_count.reset_index(drop=True)
                columns_list = df_feature_count.columns.tolist()
                if 'osmid' in columns_list:
                    columns_list.remove('osmid')
                columns_to_add = [item for item in all_attributes[f"edges_{col}"] if item not in columns_list]
                df_feature_count = df_feature_count.rename(columns={col_nm: col + "_" + col_nm for col_nm in columns_list})
                for col_to_add in columns_to_add:
                    df_feature_count[col + "_" + col_to_add] = 0

                cleaned_df.reset_index(drop=True, inplace=True)
                df_feature_count.reset_index(drop=True, inplace=True)
                cleaned_df = pd.concat([cleaned_df, df_feature_count], axis=1)
            
            cleaned_df.set_index(idx, inplace=True) 
            cleaned_df.drop(['est_width'], axis=1, inplace=True) if 'est_width' in cleaned_df.columns else None
            cols_to_drop = [col for col in cleaned_df.columns if 'unspecified' in col]
            for col in cols_to_drop:
                if col in cleaned_df.columns:
                    cleaned_df = cleaned_df.drop(col, axis=1)
            self.gdf_edges = cleaned_df
            return cleaned_df
        


#### Seattle

In [5]:
data = pd.read_csv('../data/downstream_tasks/airbnb/seattle/airbnb_seattle_cleaned.csv')
gdf_airbnb_seattle = gpd.GeoDataFrame(
            data.drop(["latitude", "longitude"], axis=1),
            geometry=gpd.points_from_xy(x=data["longitude"], y=data["latitude"]),
            crs="EPSG:4326",
        )

gdf_airbnb_seattle['airbnb'] = 1

In [6]:
G =ox.graph.graph_from_place("Seattle, United States", network_type="drive")
gdf_nodes_seattle, gdf_edges_seattle = ox.graph_to_gdfs(G)

In [7]:
graph_embedder = OSMnxGraph(gdf_airbnb_seattle, gdf_nodes_seattle, gdf_edges_seattle, y_column_name='airbnb')

osmnx_nodes = graph_embedder.get_node_attrs()
display(osmnx_nodes)
print(len(osmnx_nodes.columns.unique()))
print(osmnx_nodes.columns.unique())
osmnx_nodes.to_csv('../data/osmnx/seattle_osmnx_nodes.csv', index=False)

osmnx_edges = graph_embedder.get_edge_attrs()
display(osmnx_edges)
print(len(osmnx_edges.columns.unique()))
print(osmnx_edges.columns.unique())
osmnx_edges.to_csv('../data/osmnx/seattle_osmnx_edges.csv', index=False)



/Users/jjaniak/anaconda3/envs/studia_projekt/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,street_count,crossing,give_way,mini_roundabout,motorway_junction,stop,stop_line,traffic_signals,traffic_signals;crossing,turning_circle,...,construction,red_light_camera,bus_stop,proposed,stop;crossing,toll_gantry,speed_camera,priority,street_lamp,milestone
osmid,,,,,,,,,,,,,,,,,,,,,
29445663,3,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29447670,3,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29449421,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29449538,3,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29449863,4,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11905508650,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11905508654,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11905739466,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


22
Index(['street_count', 'crossing', 'give_way', 'mini_roundabout',
       'motorway_junction', 'stop', 'stop_line', 'traffic_signals',
       'traffic_signals;crossing', 'turning_circle', 'turning_loop',
       'passing_place', 'construction', 'red_light_camera', 'bus_stop',
       'proposed', 'stop;crossing', 'toll_gantry', 'speed_camera', 'priority',
       'street_lamp', 'milestone'],
      dtype='object')


/var/folders/4k/8js3jm9n457cm4_v631zyrkw0000gn/T/ipykernel_57856/3438227324.py:88: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  attrs = attrs.explode(col)
/Users/jjaniak/anaconda3/envs/studia_projekt/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/jjaniak/anaconda3/envs/studia_projekt/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/jjaniak/anaconda3/envs/studia_projekt/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Use

oneway  lanes maxspeed  reversed  length  width  \
u           v          key                                                      
29445663    335444348  0   0       1      2       97         0  636.02  15.17   
            29485641   0   0       1      3       97         0  688.84  15.17   
29447670    3391701882 0   0       1      2       37         0  389.72  15.17   
                           0       1      3       37         0  389.72  15.17   
                           0       1      1       37         0  389.72  15.17   
...                              ...    ...      ...       ...     ...    ...   
11913405475 53252809   0   0       0      4       40         0   35.77  15.17   
            53180224   0   0       0      4       40         1   82.34  15.17   
11914083232 53182405   0   0       0      2       32         1  103.12  15.17   
            53092559   0   0       0      2       32         0  204.48  15.17   
            53100819   0   0       0      2       32         1   98.50  15.17   

                              highway_busway  highway_living_street  \
u           v          key                                            
29445663    335444348  0   0               0                      0   
            29485641   0   0               0                      0   
29447670    3391701882 0   0               0                      0   
                           0               0                      0   
                           0               0                      0   
...                                      ...                    ...   
11913405475 53252809   0   0               0                      0   
            53180224   0   0               0                      0   
11914083232 53182405   0   0               0                      0   
            53092559   0   0               0                      0   
            53100819   0   0               0                      0   

                              highway_motorway  highway_motorway_link  ...  \
u           v          key                                             ...   
29445663    335444348  0   0                 0                      1  ...   
            29485641   0   0                 1                      0  ...   
29447670    3391701882 0   0                 0                      1  ...   
                           0                 0                      1  ...   
                           0                 0                      1  ...   
...                                        ...                    ...  ...   
11913405475 53252809   0   0                 0                      0  ...   
            53180224   0   0                 0                      0  ...   
11914083232 53182405   0   0                 0                      0  ...   
            53092559   0   0                 0                      0  ...   
            53100819   0   0                 0                      0  ...   

                              junction_roundabout  junction_jughandle  \
u           v          key                                              
29445663    335444348  0   0                    0                   0   
            29485641   0   0                    0                   0   
29447670    3391701882 0   0                    0                   0   
                           0                    0                   0   
                           0                    0                   0   
...                                           ...                 ...   
11913405475 53252809   0   0                    0                   0   
            53180224   0   0                    0                   0   
11914083232 53182405   0   0                    0                   0   
            53092559   0   0                    0                   0   
            53100819   0   0                    0                   0   

                              junction_spui  bridge_movable  bridge_viaduct  \
u       

43
Index(['oneway', 'lanes', 'maxspeed', 'reversed', 'length', 'width',
       'highway_busway', 'highway_living_street', 'highway_motorway',
       'highway_motorway_link', 'highway_primary', 'highway_primary_link',
       'highway_residential', 'highway_secondary', 'highway_secondary_link',
       'highway_stop_line', 'highway_tertiary', 'highway_tertiary_link',
       'highway_trunk', 'highway_trunk_link', 'highway_unclassified',
       'access_bus', 'access_customers', 'access_destination', 'access_no',
       'access_permissive', 'access_permit', 'access_yes', 'access_residents',
       'access_designated', 'access_delivery', 'junction_circular',
       'junction_intersection', 'junction_roundabout', 'junction_jughandle',
       'junction_spui', 'bridge_movable', 'bridge_viaduct', 'bridge_yes',
       'bridge_low_water_crossing', 'tunnel_building_passage', 'tunnel_yes',
       'tunnel_no'],
      dtype='object')


#### New York

In [8]:
data = pd.read_csv('../data/downstream_tasks/airbnb/new_york/airbnb_new_york_cleaned.csv')
gdf_airbnb_new_york = gpd.GeoDataFrame(
            data.drop(["latitude", "longitude"], axis=1),
            geometry=gpd.points_from_xy(x=data["longitude"], y=data["latitude"]),
            crs="EPSG:4326",
        )

gdf_airbnb_new_york['airbnb'] = 1

In [10]:
G =ox.graph.graph_from_place("New York City, United States", network_type="drive")
gdf_nodes_ny, gdf_edges_ny = ox.graph_to_gdfs(G)

In [12]:
graph_embedder_ny = OSMnxGraph(gdf_airbnb_new_york, gdf_nodes_ny, gdf_edges_ny, y_column_name='airbnb')

osmnx_nodes_ny = graph_embedder_ny.get_node_attrs()
display(osmnx_nodes_ny)
print(len(osmnx_nodes_ny.columns.unique()))
print(osmnx_nodes_ny.columns.unique())
osmnx_nodes_ny.to_csv('../data/osmnx/new_york_osmnx_nodes.csv', index=False)

osmnx_edges_ny = graph_embedder_ny.get_edge_attrs()
display(osmnx_edges_ny)
print(len(osmnx_edges_ny.columns.unique()))
print(osmnx_edges_ny.columns.unique())
osmnx_edges_ny.to_csv('../data/osmnx/new_york_osmnx_edges.csv', index=False)

/Users/jjaniak/anaconda3/envs/studia_projekt/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,street_count,crossing,give_way,motorway_junction,priority,stop,toll_gantry,traffic_signals,traffic_signals;crossing,turning_circle,...,construction,stop_line,red_light_camera,bus_stop,proposed,stop;crossing,speed_camera,mini_roundabout,street_lamp,milestone
osmid,,,,,,,,,,,,,,,,,,,,,
39076461,3,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39076490,3,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39076504,3,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
42421728,3,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
42421731,4,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11893511664,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11899662283,3,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11913934549,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


22
Index(['street_count', 'crossing', 'give_way', 'motorway_junction', 'priority',
       'stop', 'toll_gantry', 'traffic_signals', 'traffic_signals;crossing',
       'turning_circle', 'turning_loop', 'passing_place', 'construction',
       'stop_line', 'red_light_camera', 'bus_stop', 'proposed',
       'stop;crossing', 'speed_camera', 'mini_roundabout', 'street_lamp',
       'milestone'],
      dtype='object')


/var/folders/4k/8js3jm9n457cm4_v631zyrkw0000gn/T/ipykernel_57856/3438227324.py:88: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  attrs = attrs.explode(col)
/Users/jjaniak/anaconda3/envs/studia_projekt/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/jjaniak/anaconda3/envs/studia_projekt/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/jjaniak/anaconda3/envs/studia_projekt/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Use

oneway  reversed  length  lanes maxspeed  width  \
u           v          key                                                      
39076461    42854803   0   0       1         0  254.71      2       43  16.99   
            274283981  0   0       1         0  767.80      2       80  16.99   
                           0       1         0  767.80      2       80  16.99   
39076490    277672046  0   0       1         0  259.67      2       43  16.99   
            277672005  0   0       1         0  291.84      3       80  16.99   
...                              ...       ...     ...    ...      ...    ...   
11913934549 42844464   0   0       0         1  102.29      2       43  16.99   
11914331203 42749434   0   0       0         1   45.31      2       43  16.99   
11919089004 42916014   0   0       0         0   10.80      2       43  16.99   
            6864033185 0   0       0         1  101.33      2       43  16.99   
            4163728666 0   0       1         0  101.50      2       43  16.99   

                              highway_living_street  highway_motorway  \
u           v          key                                              
39076461    42854803   0   0                      0                 0   
            274283981  0   0                      0                 1   
                           0                      0                 1   
39076490    277672046  0   0                      0                 0   
            277672005  0   0                      0                 1   
...                                             ...               ...   
11913934549 42844464   0   0                      0                 0   
11914331203 42749434   0   0                      0                 0   
11919089004 42916014   0   0                      0                 0   
            6864033185 0   0                      0                 0   
            4163728666 0   0                      0                 0   

                              highway_motorway_link  highway_primary  ...  \
u           v          key                                            ...   
39076461    42854803   0   0                      1                0  ...   
            274283981  0   0                      0                0  ...   
                           0                      0                0  ...   
39076490    277672046  0   0                      1                0  ...   
            277672005  0   0                      0                0  ...   
...                                             ...              ...  ...   
11913934549 42844464   0   0                      0                0  ...   
11914331203 42749434   0   0                      0                0  ...   
11919089004 42916014   0   0                      0                0  ...   
            6864033185 0   0                      0                0  ...   
            4163728666 0   0                      0                1  ...   

                              junction_roundabout  junction_spui  \
u           v          key                                         
39076461    42854803   0   0                    0              0   
            274283981  0   0                    0              0   
                           0                    0              0   
39076490    277672046  0   0                    0              0   
            277672005  0   0                    0              0   
...                                           ...            ...   
11913934549 42844464   0   0                    0              0   
11914331203 42749434   0   0                    0              0   
11919089004 42916014   0   0                    0              0   
            6864033185 0   0                    0              0   
            4163728666 0   0                    0              0   

                              junction_intersection  bridge_movable  \
u           v          key                                          

43
Index(['oneway', 'reversed', 'length', 'lanes', 'maxspeed', 'width',
       'highway_living_street', 'highway_motorway', 'highway_motorway_link',
       'highway_primary', 'highway_primary_link', 'highway_residential',
       'highway_secondary', 'highway_secondary_link', 'highway_tertiary',
       'highway_tertiary_link', 'highway_trunk', 'highway_trunk_link',
       'highway_unclassified', 'highway_busway', 'highway_stop_line',
       'access_customers', 'access_designated', 'access_destination',
       'access_no', 'access_permissive', 'access_permit', 'access_yes',
       'access_residents', 'access_bus', 'access_delivery',
       'junction_circular', 'junction_jughandle', 'junction_roundabout',
       'junction_spui', 'junction_intersection', 'bridge_movable',
       'bridge_viaduct', 'bridge_yes', 'bridge_low_water_crossing',
       'tunnel_building_passage', 'tunnel_no', 'tunnel_yes'],
      dtype='object')


## PL

In [29]:
city = 'Szczecin'

df_accidents = pd.read_csv("../data/wypadki-pl/accidents.csv")
df_accidents.drop(df_accidents[(df_accidents['mie_nazwa'] != city)].index, inplace=True)
df_accidents.drop(columns='uczestnicy', inplace=True)
geometry = [Point(xy) for xy in zip(df_accidents['wsp_gps_x'], df_accidents['wsp_gps_y'])]
gdf_accidents = gpd.GeoDataFrame(df_accidents, geometry=geometry)
gdf_accidents.drop(columns=['wsp_gps_x', 'wsp_gps_y'], inplace=True)

In [30]:
cs = '[out:json][timeout:180][date:"2022-01-01T00:00:00Z"]'
ox.settings.overpass_settings = cs
G =ox.graph.graph_from_place(f"{city}, Polska", network_type="drive")
gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)

In [31]:
graph_embedder = OSMnxGraph(gdf_airbnb_seattle, gdf_nodes_seattle, gdf_edges_seattle, y_column_name='mie_nazwa')

osmnx_nodes = graph_embedder.get_node_attrs()
display(osmnx_nodes)
print(len(osmnx_nodes.columns.unique()))
print(osmnx_nodes.columns.unique())
osmnx_nodes.to_csv(f'../data/osmnx/{city}_osmnx_nodes.csv', index=False)

osmnx_edges = graph_embedder.get_edge_attrs()
display(osmnx_edges)
print(len(osmnx_edges.columns.unique()))
print(osmnx_edges.columns.unique())
osmnx_edges.to_csv(f'../data/osmnx/{city}_osmnx_edges.csv', index=False)


/Users/jjaniak/anaconda3/envs/studia_projekt/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,street_count,crossing,give_way,mini_roundabout,motorway_junction,stop,stop_line,traffic_signals,traffic_signals;crossing,turning_circle,...,construction,red_light_camera,bus_stop,proposed,stop;crossing,toll_gantry,speed_camera,priority,street_lamp,milestone
osmid,,,,,,,,,,,,,,,,,,,,,
29445663,3,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29447670,3,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29449421,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29449538,3,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29449863,4,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11905508650,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11905508654,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11905739466,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


22
Index(['street_count', 'crossing', 'give_way', 'mini_roundabout',
       'motorway_junction', 'stop', 'stop_line', 'traffic_signals',
       'traffic_signals;crossing', 'turning_circle', 'turning_loop',
       'passing_place', 'construction', 'red_light_camera', 'bus_stop',
       'proposed', 'stop;crossing', 'toll_gantry', 'speed_camera', 'priority',
       'street_lamp', 'milestone'],
      dtype='object')


/var/folders/4k/8js3jm9n457cm4_v631zyrkw0000gn/T/ipykernel_57856/3438227324.py:88: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  attrs = attrs.explode(col)
/Users/jjaniak/anaconda3/envs/studia_projekt/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/jjaniak/anaconda3/envs/studia_projekt/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/jjaniak/anaconda3/envs/studia_projekt/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Use

oneway  lanes maxspeed  reversed  length  width  \
u           v          key                                                      
29445663    335444348  0   0       1      2       97         0  636.02  15.17   
            29485641   0   0       1      3       97         0  688.84  15.17   
29447670    3391701882 0   0       1      2       37         0  389.72  15.17   
                           0       1      3       37         0  389.72  15.17   
                           0       1      1       37         0  389.72  15.17   
...                              ...    ...      ...       ...     ...    ...   
11913405475 53252809   0   0       0      4       40         0   35.77  15.17   
            53180224   0   0       0      4       40         1   82.34  15.17   
11914083232 53182405   0   0       0      2       32         1  103.12  15.17   
            53092559   0   0       0      2       32         0  204.48  15.17   
            53100819   0   0       0      2       32         1   98.50  15.17   

                              highway_busway  highway_living_street  \
u           v          key                                            
29445663    335444348  0   0               0                      0   
            29485641   0   0               0                      0   
29447670    3391701882 0   0               0                      0   
                           0               0                      0   
                           0               0                      0   
...                                      ...                    ...   
11913405475 53252809   0   0               0                      0   
            53180224   0   0               0                      0   
11914083232 53182405   0   0               0                      0   
            53092559   0   0               0                      0   
            53100819   0   0               0                      0   

                              highway_motorway  highway_motorway_link  ...  \
u           v          key                                             ...   
29445663    335444348  0   0                 0                      1  ...   
            29485641   0   0                 1                      0  ...   
29447670    3391701882 0   0                 0                      1  ...   
                           0                 0                      1  ...   
                           0                 0                      1  ...   
...                                        ...                    ...  ...   
11913405475 53252809   0   0                 0                      0  ...   
            53180224   0   0                 0                      0  ...   
11914083232 53182405   0   0                 0                      0  ...   
            53092559   0   0                 0                      0  ...   
            53100819   0   0                 0                      0  ...   

                              junction_roundabout  junction_jughandle  \
u           v          key                                              
29445663    335444348  0   0                    0                   0   
            29485641   0   0                    0                   0   
29447670    3391701882 0   0                    0                   0   
                           0                    0                   0   
                           0                    0                   0   
...                                           ...                 ...   
11913405475 53252809   0   0                    0                   0   
            53180224   0   0                    0                   0   
11914083232 53182405   0   0                    0                   0   
            53092559   0   0                    0                   0   
            53100819   0   0                    0                   0   

                              junction_spui  bridge_movable  bridge_viaduct  \
u       

43
Index(['oneway', 'lanes', 'maxspeed', 'reversed', 'length', 'width',
       'highway_busway', 'highway_living_street', 'highway_motorway',
       'highway_motorway_link', 'highway_primary', 'highway_primary_link',
       'highway_residential', 'highway_secondary', 'highway_secondary_link',
       'highway_stop_line', 'highway_tertiary', 'highway_tertiary_link',
       'highway_trunk', 'highway_trunk_link', 'highway_unclassified',
       'access_bus', 'access_customers', 'access_destination', 'access_no',
       'access_permissive', 'access_permit', 'access_yes', 'access_residents',
       'access_designated', 'access_delivery', 'junction_circular',
       'junction_intersection', 'junction_roundabout', 'junction_jughandle',
       'junction_spui', 'bridge_movable', 'bridge_viaduct', 'bridge_yes',
       'bridge_low_water_crossing', 'tunnel_building_passage', 'tunnel_yes',
       'tunnel_no'],
      dtype='object')
